## Notebook for spike sorting from .kwd data using the Pipeline of mountainsort (linux channel has pipeline)
Uses:
    - intan2kwik (https://github.com/zekearneodo/intan2kwik/blob/master/README.md)
    - mountainlab suite(https://github.com/flatironinstitute/mountainlab-js)
    - mountainsort https://github.com/flatironinstitute/mountainsort_examples/blob/master/README.md
    - mountainsort examples https://github.com/flatironinstitute/mountainsort_examples/blob/master/README.md

In [1]:
import socket
import os
import glob
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import subprocess
from datetime import timedelta
from importlib import reload

# pipeline imports
from pipefinch.neural.convert import intan
from pipefinch.neural.sort.mountain import core as msc
from pipefinch.h5tools.kwik import kutil
from pipefinch.pipeline import probes


from pipefinch.h5tools.kwik import kwdfunctions as kwdf

from intan2kwik import kwd

#mountainsort imports (for sorting)
#import mountainlab_pytools.mlproc as mlp

import logging

# Setup the logger
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

ch = logging.StreamHandler()
ch.setLevel(logging.INFO)
ch.setFormatter(formatter)
logger.addHandler(ch)
        
logger.info('Logger set')
logger.info('Hostname {}'.format(socket.gethostname()))

2019-06-11 23:39:45,764 - root - INFO - Logger set
2019-06-11 23:39:45,765 - root - INFO - Hostname lookfar


In [2]:
## ad hoc locations dict
locations_dict  = {'mnt': os.path.abspath('/Users/zeke/experiment'),
                             'local': os.path.abspath('/Users/zeke/experiment'),
                             'fast': os.path.abspath('/Users/zeke/experiment')
                             }

### Session parameters and raw files

In [3]:
from pipefinch.pipeline import filestructure as et
reload(et)
reload(kwd)

sess_par = {'bird': 'g19o19',
           'sess': '500a_2500l_3400v',
           'probe': 'probe_0', # probe to sort ('probe_0', 'probe_1') (to lookup in the rig_par which port to extract)
            'sort': 0
           }

exp_struct = et.get_exp_struct(sess_par['bird'], sess_par['sess'], sess_par['sort'], location_dict=locations_dict)

# mountainsort parameters
sort_params = {'adjacency_radius': -1,
              'detect_threshold': 2,
              'freq_min': 600}

# differetn mountainsort parameters
ds_params = {'detect_sign': -1}


# convenient paths
kwik_folder = exp_struct['folders']['kwik']
msort_folder = exp_struct['folders']['msort']
raw_folder = exp_struct['folders']['raw']
kwd_path = exp_struct['files']['kwd']
bin_path = exp_struct['files']['mda_raw']

In [4]:
exp_struct

{'folders': {'raw': '/Users/zeke/experiment/g19o19/Ephys/raw/500a_2500l_3400v',
  'kwik': '/Users/zeke/experiment/g19o19/Ephys/kwik/500a_2500l_3400v',
  'msort': '/Users/zeke/experiment/g19o19/Ephys/msort/500a_2500l_3400v',
  'ksort': '/Users/zeke/experiment/g19o19/Ephys/ksort/500a_2500l_3400v'},
 'files': {'par': '/Users/zeke/experiment/g19o19/Ephys/ksort/500a_2500l_3400v/params.json',
  'set': '/Users/zeke/experiment/g19o19/Ephys/raw/500a_2500l_3400v/settings.isf',
  'rig': '/Users/zeke/experiment/g19o19/Ephys/raw/500a_2500l_3400v/rig.json',
  'kwd': '/Users/zeke/experiment/g19o19/Ephys/kwik/500a_2500l_3400v/streams.kwd',
  'kwik': '/Users/zeke/experiment/g19o19/Ephys/kwik/500a_2500l_3400v/sort_0/spikes.kwik',
  'kwe': '/Users/zeke/experiment/g19o19/Ephys/kwik/500a_2500l_3400v/events.kwe',
  'mda_raw': '/Users/zeke/experiment/g19o19/Ephys/msort/500a_2500l_3400v/raw.mda',
  'bin_raw': '/Users/zeke/experiment/g19o19/Ephys/ksort/500a_2500l_3400v/raw.bin'}}

### convert the whole session to a .kwd file
Conversion sends every .rhd file in the folder to a rec in the .kwd file (experiment.kwd in the session ss folder)
All of the files and all of the channels are converted; filtering and subselection of sub-epochs and channels occurs later.
The .kwd is raw data, only in a friendlier format.


#### Make a file for the session for the first time

In [5]:
reload(kwd)
## Convert the whole session to a kwd file
os.makedirs(kwik_folder, exist_ok=True)
first_intan_hdr, sess_pd = kwd.intan_to_kwd(raw_folder, kwd_path, overwrite=False)

2019-06-11 23:39:47,352 - intan2kwik.kwd - WARNING - File /Users/zeke/experiment/g19o19/Ephys/kwik/500a_2500l_3400v/streams.kwd already exists
2019-06-11 23:39:47,353 - intan2kwik.kwd - WARNING - Will abort. To overwrite the existing file, use overwrite=True option


RuntimeError: Kwd file already exists

#### Update a session with subsequently recorded rhd files

In [6]:
reload(kwdf)
_, nu_pd, _ = kwdf.update_kwd(kwd_path, raw_folder)

2019-06-11 23:39:53,491 - pipefinch.h5tools.kwik.kwdfunctions - INFO - updating kwd file /Users/zeke/experiment/g19o19/Ephys/kwik/500a_2500l_3400v/streams.kwd from folder /Users/zeke/experiment/g19o19/Ephys/raw/500a_2500l_3400v
2019-06-11 23:39:53,529 - pipefinch.h5tools.kwik.kwdfunctions - INFO - No new files to add to the file


## Make .mda file with a set of recordings in a session
 - pick all in port A
 - get all rec within a time range


In [7]:
# get the session meta
pd_meta = kwdf.get_all_rec_meta(kwd_path)
pd_meta.head(2)


,bit_depth,name,sample_rate,start_sample,start_time,channel_bit_volts,channel_names,channels_sample_rate,is_multiSampleRate_data,valid_samples,samples_count
0,16,0,20000.0,0,2019-06-07 18:21:47,"[0.195, 0.195, 0.195, 0.195, 0.195, 0.195, 0.1...","[A-000, A-001, A-002, A-003, A-004, A-005, A-0...","[20000.0, 20000.0, 20000.0, 20000.0, 20000.0, ...",0,"[8391600.0, 8391600.0, 8391600.0, 8391600.0, 8...",8391600


### pick a time interval of the recordings

In [8]:
def select_time_span(meta_pd, start, span_minutes):
    end = start + timedelta(minutes=span_minutes)
    pd_selection = meta_pd.loc[meta_pd['start_time'].between(start, end)]
    return pd_selection

pd_meta_selection = select_time_span(pd_meta, pd_meta['start_time'][0]+ timedelta(minutes=0*12), 240)

In [9]:
pd_meta['start_time'].max() - pd_meta['start_time'].min()

Timedelta('0 days 00:00:00')

In [10]:
# for instance
pd_meta_selection.index

Int64Index([0], dtype='int64')

In [11]:
# load the rig parameters
rig_par_file = exp_struct['files']['rig']
with open(rig_par_file, 'r') as fp:
    rig_par = json.load(fp)

# get the probe and the port where the probe was connected
selected_probe = sess_par['probe']
probe_port = rig_par['chan']['port'][selected_probe].strip('-')

# get the channel indices of the probe's port
wanted_chans = np.array([probe_port + '-'])  # all ephys channels

chan_list = kwdf.get_all_chan_names(pd_meta_selection, chan_filt=wanted_chans)

#all_rec_list = kutil.get_rec_list(exp_struct['files']['kwd'])
selection_rec_list = np.unique(pd_meta_selection['name'])

rec_chans = pd_meta.loc[pd_meta['name'] == selection_rec_list[0], 'channel_names'].values
rec_chans_idx = kwdf.find_chan_names_idx(rec_chans[0], chan_list)

# make the mda binary file
bin_path = exp_struct['files']['mda_raw']
os.makedirs(exp_struct['folders']['msort'], exist_ok=True)
bin_file = kwdf.kwd_to_binary(exp_struct['files']['kwd'],
                              exp_struct['files']['mda_raw'],
                              chan_list=chan_list,
                              rec_list=selection_rec_list, header='mda')


2019-06-11 23:39:59,291 - pipefinch.h5tools.kwik.kwdfunctions - INFO - Writing kwd_file /Users/zeke/experiment/g19o19/Ephys/kwik/500a_2500l_3400v/streams.kwd to binary
2019-06-11 23:39:59,298 - pipefinch.h5tools.kwik.kwdfunctions - INFO - Channels to extract: ['A-000' 'A-001' 'A-002' 'A-003' 'A-004' 'A-005' 'A-006' 'A-007' 'A-008'
 'A-009' 'A-011' 'A-012' 'A-013' 'A-014' 'A-015' 'A-016' 'A-017' 'A-018'
 'A-019' 'A-020' 'A-021' 'A-022' 'A-023' 'A-024' 'A-025' 'A-026' 'A-027'
 'A-028' 'A-029' 'A-030' 'A-031' 'A-032' 'A-033' 'A-034' 'A-035' 'A-036'
 'A-037' 'A-038' 'A-039' 'A-040' 'A-041' 'A-042' 'A-043' 'A-044' 'A-045'
 'A-046' 'A-047' 'A-048' 'A-049' 'A-050' 'A-051' 'A-052' 'A-053' 'A-054'
 'A-055' 'A-056' 'A-057' 'A-058' 'A-059' 'A-060' 'A-061' 'A-062' 'A-063']
2019-06-11 23:39:59,300 - pipefinch.h5tools.kwik.kwdfunctions - INFO - Will go through recs [0]
2019-06-11 23:39:59,301 - pipefinch.h5tools.kwik.kwdfunctions - INFO - Creating binary file /Users/zeke/experiment/g19o19/Ephys/msor

2019-06-11 23:40:11,156 - pipefinch.h5tools.kwik.kwdfunctions - INFO - 528670800 elements written


In [12]:
rec_chans

array([array(['A-000', 'A-001', 'A-002', 'A-003', 'A-004', 'A-005', 'A-006',
       'A-007', 'A-008', 'A-009', 'A-011', 'A-012', 'A-013', 'A-014',
       'A-015', 'A-016', 'A-017', 'A-018', 'A-019', 'A-020', 'A-021',
       'A-022', 'A-023', 'A-024', 'A-025', 'A-026', 'A-027', 'A-028',
       'A-029', 'A-030', 'A-031', 'A-032', 'A-033', 'A-034', 'A-035',
       'A-036', 'A-037', 'A-038', 'A-039', 'A-040', 'A-041', 'A-042',
       'A-043', 'A-044', 'A-045', 'A-046', 'A-047', 'A-048', 'A-049',
       'A-050', 'A-051', 'A-052', 'A-053', 'A-054', 'A-055', 'A-056',
       'A-057', 'A-058', 'A-059', 'A-060', 'A-061', 'A-062', 'A-063'],
      dtype='<U5')], dtype=object)

## Scripts for sorting

### prep the files with their nice formats, locations and names


In [13]:
reload(msc)
file_paths, out_folder = msc.make_paths(exp_struct['folders']['msort'])
os.makedirs(exp_struct['folders']['msort'], exist_ok=True)

# make the probe file
rec_chans = pd_meta.loc[pd_meta['name']==0, 'channel_names'].values
rec_chans_idx = kwdf.find_chan_names_idx(rec_chans[0], chan_list)
probe = rig_par['probe'][selected_probe]['model']
headstage = rig_par['probe'][selected_probe]['headstage']
probe_chans = rec_chans_idx - np.min(rec_chans_idx)
# try to make a probe. If it is not possible, force adjacency_radius to -1.
try:
    probe_geom = probes.make_map(probe, probe_chans)
    np.savetxt(file_paths['geom'], probe_geom, delimiter=',')
except KeyError as err:
    logger.warning('Probe could not be made, probe or headstage not found {} in probes.py. Will sort with no geometry'.format(err))
    sort_params['adjacency_radius'] = -1

# parameters to pass to the msort scripts
ds_params.update({'samplerate': int(kwdf.get_sampling_rate(pd_meta, 0)), # required
            })


with open(file_paths['params'], 'w') as fp:
    json.dump(ds_params, fp)
    logger.info('Created session par files {}'.format(file_paths['params']))


2019-06-11 23:48:09,621 - root - WARNING - Probe could not be made, probe or headstage not found 'ASSY-236-H5' in probes.py. Will sort with no geometry
2019-06-11 23:48:09,624 - root - INFO - Created session par files /Users/zeke/experiment/g19o19/Ephys/msort/500a_2500l_3400v/params.json


In [14]:
file_paths

{'mda': '/Users/zeke/experiment/g19o19/Ephys/msort/500a_2500l_3400v/raw.mda',
 'params': '/Users/zeke/experiment/g19o19/Ephys/msort/500a_2500l_3400v/params.json',
 'geom': '/Users/zeke/experiment/g19o19/Ephys/msort/500a_2500l_3400v/geom.csv',
 'filt': '/Users/zeke/experiment/g19o19/Ephys/msort/500a_2500l_3400v/sort_out/filt.mda.prv',
 'pre': '/Users/zeke/experiment/g19o19/Ephys/msort/500a_2500l_3400v/sort_out/pre.mda.prv',
 'firings': '/Users/zeke/experiment/g19o19/Ephys/msort/500a_2500l_3400v/sort_out/firings.mda',
 'firings_curated': '/Users/zeke/experiment/g19o19/Ephys/msort/500a_2500l_3400v/sort_out/firings_curated.mda',
 'cluster_metrics': '/Users/zeke/experiment/g19o19/Ephys/msort/500a_2500l_3400v/sort_out/cluster_metrics.json',
 'cluster_metrics_curated': '/Users/zeke/experiment/g19o19/Ephys/msort/500a_2500l_3400v/sort_out/cluster_metrics_curated.json'}

In [15]:
# test mountainsort method for reading sorting parameters
msc.read_dataset_params(exp_struct['folders']['msort'])

{'detect_sign': -1, 'samplerate': 20000}

In [16]:
output_dir = os.path.join(exp_struct['folders']['msort'], 'sort_out');
msc.sort_dataset(file_paths=file_paths, **sort_params);

2019-06-11 23:48:14,122 - pipefinch.sort.mountain.comre - INFO - Bandpass filter


RUNNING: ml-run-process ephys.bandpass_filter --inputs timeseries:/Users/zeke/experiment/g19o19/Ephys/msort/500a_2500l_3400v/raw.mda --parameters freq_max:6000 freq_min:600 samplerate:20000 --outputs timeseries_out:/Users/zeke/experiment/g19o19/Ephys/msort/500a_2500l_3400v/sort_out/filt.mda.prv


2019-06-11 23:48:32,241 - pipefinch.sort.mountain.comre - INFO - Whitening


[ Getting processor spec... ]
[ Checking inputs and substituting prvs ... ]
[ Computing process signature ... ]
[ Checking outputs... ]
[ Checking process cache ... ]
[ Creating temporary directory ... ]
[ Preparing temporary outputs... ]
[ Initializing process ... ]
[ Running ... ] /Users/zeke/anaconda3/envs/mountain/bin/python3 /Users/zeke/anaconda3/envs/mountain/etc/mountainlab/packages/ml_ephys/preprocessing/preprocessing.py.mp ephys.bandpass_filter --timeseries=/Users/zeke/experiment/g19o19/Ephys/msort/500a_2500l_3400v/raw.mda --timeseries_out=/tmp/mountainlab-tmp/tempdir_bfa348a926_4T5xph/output_timeseries_out.mda --freq_max=6000 --freq_min=600 --samplerate=20000 --_tempdir=/tmp/mountainlab-tmp/tempdir_bfa348a926_4T5xph
Chunk size: 30000, Padding: 3000, Num chunks: 280, Num processes: 12

Processed 160 of 280 chunks...

Processed 272 of 280 chunks...
Processed 71 of 280 chunks...

Elapsed time for processor ephys.bandpass_filter: 12.893 sec
Finalizing output timeseries_out
[ Crea

2019-06-11 23:48:48,703 - pipefinch.sort.mountain.comre - INFO - Sorting


...

RUNNING: ml-run-process ms4alg.sort --inputs timeseries:/Users/zeke/experiment/g19o19/Ephys/msort/500a_2500l_3400v/sort_out/pre.mda.prv --parameters adjacency_radius:-1 detect_sign:-1 detect_threshold:2 --outputs firings_out:/Users/zeke/experiment/g19o19/Ephys/msort/500a_2500l_3400v/sort_out/firings.mda


2019-06-12 00:03:51,440 - pipefinch.sort.mountain.comre - INFO - Getting cluster metrics


[ Getting processor spec... ]
[ Checking inputs and substituting prvs ... ]
Locating /Users/zeke/experiment/g19o19/Ephys/msort/500a_2500l_3400v/sort_out/pre.mda.prv ...
[ Computing process signature ... ]
[ Checking outputs... ]
[ Checking process cache ... ]
[ Creating temporary directory ... ]
[ Preparing temporary outputs... ]
[ Initializing process ... ]
[ Running ... ] /Users/zeke/anaconda3/envs/mountain/bin/python3 /Users/zeke/anaconda3/envs/mountain/etc/mountainlab/packages/ml_ms4alg/ms4alg_spec.py.mp ms4alg.sort --timeseries=/tmp/mountainlab-tmp/output_663d8cd52c73b64296dbac22b3023bb16b1f44b7_timeseries_out.mda --geom= --firings_out=/tmp/mountainlab-tmp/tempdir_cadcce5f95_JWxJ4H/output_firings_out.mda --adjacency_radius=-1 --detect_sign=-1 --detect_threshold=2 --_tempdir=/tmp/mountainlab-tmp/tempdir_cadcce5f95_JWxJ4H
Using tempdir=/tmp/mountainlab-tmp/tempdir_cadcce5f95_JWxJ4H
Preparing /tmp/mountainlab-tmp/tempdir_cadcce5f95_JWxJ4H/timeseries.hdf5...
Preparing neighborhood sor

2019-06-12 00:05:52,795 - pipefinch.sort.mountain.comre - INFO - Automatically curating


[ Getting processor spec... ]
[ Checking inputs and substituting prvs ... ]
Locating /tmp/mountainlab-tmp/mountainlab/tmp_short_term/output_cluster_metrics_out_0402d25e4c360dafd28a7906b8e4299b85c133b6.prv ...
Locating /tmp/mountainlab-tmp/mountainlab/tmp_short_term/output_metrics_out_f152da302a86135760e57614593963a3f1a5467b.prv ...
[ Computing process signature ... ]
[ Checking outputs... ]
[ Checking process cache ... ]
[ Creating temporary directory ... ]
[ Preparing temporary outputs... ]
[ Initializing process ... ]
[ Running ... ] /Users/zeke/anaconda3/envs/mountain/etc/mountainlab/packages/ms3.mp ms3.combine_cluster_metrics --metrics_list=/tmp/mountainlab-tmp/output_8fda44fe58afedecc8f27584a807da29022403d8_cluster_metrics_out --metrics_list=/tmp/mountainlab-tmp/output_f116eac2d56164e0d03b0841de57b13855d24d2a_metrics_out --metrics_out=/tmp/mountainlab-tmp/tempdir_bdb7056d28_8LHs5R/output_metrics_out.json --_tempdir=/tmp/mountainlab-tmp/tempdir_bdb7056d28_8LHs5R
Elapsed time for pr

In [17]:
file_paths

{'mda': '/Users/zeke/experiment/g19o19/Ephys/msort/500a_2500l_3400v/raw.mda',
 'params': '/Users/zeke/experiment/g19o19/Ephys/msort/500a_2500l_3400v/params.json',
 'geom': '/Users/zeke/experiment/g19o19/Ephys/msort/500a_2500l_3400v/geom.csv',
 'filt': '/Users/zeke/experiment/g19o19/Ephys/msort/500a_2500l_3400v/sort_out/filt.mda.prv',
 'pre': '/Users/zeke/experiment/g19o19/Ephys/msort/500a_2500l_3400v/sort_out/pre.mda.prv',
 'firings': '/Users/zeke/experiment/g19o19/Ephys/msort/500a_2500l_3400v/sort_out/firings.mda',
 'firings_curated': '/Users/zeke/experiment/g19o19/Ephys/msort/500a_2500l_3400v/sort_out/firings_curated.mda',
 'cluster_metrics': '/Users/zeke/experiment/g19o19/Ephys/msort/500a_2500l_3400v/sort_out/cluster_metrics.json',
 'cluster_metrics_curated': '/Users/zeke/experiment/g19o19/Ephys/msort/500a_2500l_3400v/sort_out/cluster_metrics_curated.json'}

## Command for viewing:
 - open up terminal with the environment msort
 - go go the ss_data folder for the session
 - run the command: qt-mountainview --raw raw.mda --filt sort_out/filt.mda.prv --pre sort_out/pre.mda.prv --samplerate=20000 --firings sort_out/firings.mda --cluster_metrics sort_out/cluster_metrics.json

# After manual curation
 - save the curated spikes in the sort_out/firings_curated.mda
 - come back to the notebook and run 

In [33]:
from pipefinch.h5tools.kwik import kwikfunctions as kwkf
reload(kwkf)
reload(et)
firings_to_save = 'firings_curated' # 'curated' or 'firings' for default_output

metrics_to_save = 'cluster_metrics_curated' if firings_to_save == 'firings_curated' else 'cluster_metrics'
kwkf.mda_to_kwik(exp_struct['files']['kwd'],
                 exp_struct['files']['kwik'],
                 file_paths[firings_to_save],
                file_paths[metrics_to_save], 
                rec_in_binary=selection_rec_list)


2019-06-11 19:31:21,104 - pipefinch.h5tools.kwik.kwikfunctions - INFO - Creating kwik file /Users/zeke/experiment/g19o19/Ephys/kwik/350a_2500r_2780v/sort_0/spikes.kwik from firings /Users/zeke/experiment/g19o19/Ephys/msort/350a_2500r_2780v/sort_out/firings_curated.mda


In [34]:
selection_rec_list

array([0, 1])

In [35]:
### extract all unit waveforms
from pipefinch.neural import units
reload(units)
units.get_all_unit_waveforms(exp_struct['files']['kwik'], exp_struct['files']['kwd'], port=probe_port)

2019-06-11 19:31:25,130 - pipefinch.neural.units - INFO - About to get all waveforms for 170 units in file /Users/zeke/experiment/g19o19/Ephys/kwik/350a_2500r_2780v/sort_0/spikes.kwik


0

#### cleanup the intermediate sort files and the mountainsort .mda files

In [36]:
reload(et)
reload(msc)
et.msort_cleanup(exp_struct)
msc.clean_tmp_dir()

2019-06-11 19:31:38,269 - pipefinch.pipeline.filestructure - INFO - removing (if exists) msort mda file /Users/zeke/experiment/g19o19/Ephys/msort/350a_2500r_2780v/raw.mda 
2019-06-11 19:31:38,668 - pipefinch.sort.mountain.comre - INFO - Cleaning up msort temp dir /tmp/mountainlab-tmp


In [37]:
 exp_struct['files']

{'par': '/Users/zeke/experiment/g19o19/Ephys/ksort/350a_2500r_2780v/params.json',
 'set': '/Users/zeke/experiment/g19o19/Ephys/raw/350a_2500r_2780v/settings.isf',
 'rig': '/Users/zeke/experiment/g19o19/Ephys/raw/350a_2500r_2780v/rig.json',
 'kwd': '/Users/zeke/experiment/g19o19/Ephys/kwik/350a_2500r_2780v/streams.kwd',
 'kwik': '/Users/zeke/experiment/g19o19/Ephys/kwik/350a_2500r_2780v/sort_0/spikes.kwik',
 'kwe': '/Users/zeke/experiment/g19o19/Ephys/kwik/350a_2500r_2780v/events.kwe',
 'mda_raw': '/Users/zeke/experiment/g19o19/Ephys/msort/350a_2500r_2780v/raw.mda',
 'bin_raw': '/Users/zeke/experiment/g19o19/Ephys/ksort/350a_2500r_2780v/raw.bin'}